## Importing necessary Python libraries

In [1]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pyLDAvis.enable_notebook()

## Reading the datasets.

In [3]:
d = pd.read_excel('FitBit Reviews.xlsx', parse_dates = True)

## Keeping a copy of the original dataset, just in case.

In [4]:
d1 = d.copy()

In [5]:
d.head()

,Unnamed: 0,Activity,Acivity_General,TimeStamp,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,...,neg,neu,pos,compound,nb_chars,nb_words,Samsung Gear Mentioned,Mi Fit Mentioned,Compound Sentiment,App
0,0,2 Star Review for Fit Bit,2 Star Review,2021-03-05 00:16:09,gp:AOqpTOFFj5bsQBu0epZpqnuLSrD7xVIDfO1TKoXrdg1...,Joseph Kelly,"They may have done a critical update, now all ...",2,0,3.39.2,...,0.223,0.777,0.000,-0.3182,102,23,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Negative,FitBit
1,1,5 Star Review for Fit Bit,5 Star Review,2021-03-05 00:12:25,gp:AOqpTOEdlEMiDOYgjzrC2-ds_7D5r9JLnIeE9edUZjf...,Ken Bradford,Good app. Very motivating. Great update.,5,0,3.39.1,...,0.000,0.172,0.828,0.8625,40,6,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Positive,FitBit
2,2,3 Star Review for Fit Bit,3 Star Review,2021-03-05 00:07:48,gp:AOqpTOEOm4wYr4oV-0q2o-4yIE-33U0Ib-Kkq6GOGtR...,248baumer X,Always room.for improvement,3,0,NaN,...,0.000,0.400,0.600,0.4588,27,3,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Positive,FitBit
3,3,1 Star Review for Fit Bit,1 Star Review,2021-03-05 00:04:48,gp:AOqpTOHu0-MdLKPOwF4rv3cqvDclAbSAjNqd79-4wY8...,danny weil,Fix the bug please and don't forget and I can'...,1,3,3.39.1,...,0.084,0.711,0.204,0.4019,201,43,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Positive,FitBit
4,4,4 Star Review for Fit Bit,4 Star Review,2021-03-05 00:02:57,gp:AOqpTOFFhYm20tznHWWB0jcPZEPEmWeC7_u-3-pHUeA...,Elliott Dudnik,Works well good inf,4,0,3.39.1,...,0.000,0.286,0.714,0.6124,19,4,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Positive,FitBit


## Dropping the identifier columns

In [6]:
d.drop(['Unnamed: 0', 'reviewId', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt'], axis = 1, inplace = True)

In [7]:
d.head()

,Activity,Acivity_General,TimeStamp,userName,content,score,thumbsUpCount,Clean Titles,neg,neu,pos,compound,nb_chars,nb_words,Samsung Gear Mentioned,Mi Fit Mentioned,Compound Sentiment,App
0,2 Star Review for Fit Bit,2 Star Review,2021-03-05 00:16:09,Joseph Kelly,"They may have done a critical update, now all ...",2,0,may do critical update get fitbit time need watch,0.223,0.777,0.000,-0.3182,102,23,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Negative,FitBit
1,5 Star Review for Fit Bit,5 Star Review,2021-03-05 00:12:25,Ken Bradford,Good app. Very motivating. Great update.,5,0,good app motivate great update,0.000,0.172,0.828,0.8625,40,6,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Positive,FitBit
2,3 Star Review for Fit Bit,3 Star Review,2021-03-05 00:07:48,248baumer X,Always room.for improvement,3,0,always room.for improvement,0.000,0.400,0.600,0.4588,27,3,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Positive,FitBit
3,1 Star Review for Fit Bit,1 Star Review,2021-03-05 00:04:48,danny weil,Fix the bug please and don't forget and I can'...,1,3,fix bug please forget can't syce phone fix wor...,0.084,0.711,0.204,0.4019,201,43,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Positive,FitBit
4,4 Star Review for Fit Bit,4 Star Review,2021-03-05 00:02:57,Elliott Dudnik,Works well good inf,4,0,work well good inf,0.000,0.286,0.714,0.6124,19,4,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Positive,FitBit


## Shape of the data

In [8]:
d.shape

(306614, 18)

## Basic description of the dataset

In [9]:
d.describe(include = 'all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
Activity,306614,6,5 Star Review for Fit Bit,102530,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Acivity_General,306614,6,5 Star Review,102530,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TimeStamp,306614,297003,2016-08-09 20:41:57,4,2012-03-17 13:49:39,2021-03-05 00:16:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
userName,306614,236256,A Google user,19180,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
content,306496,272274,Love it,1982,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
score,306614,NaN,NaN,NaN,NaT,NaT,2.87731,1.88464,0,1,3,5,5
thumbsUpCount,306614,NaN,NaN,NaN,NaT,NaT,0.978886,12.1569,0,0,0,0,1308
Clean Titles,300230,220522,love,5647,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
neg,306614,NaN,NaN,NaN,NaT,NaT,0.0680785,0.134026,0,0,0,0.1,1
neu,306614,NaN,NaN,NaN,NaT,NaT,0.501021,0.334259,0,0.215,0.548,0.754,1


In [10]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306614 entries, 0 to 306613
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Activity                306614 non-null  object        
 1   Acivity_General         306614 non-null  object        
 2   TimeStamp               306614 non-null  datetime64[ns]
 3   userName                306614 non-null  object        
 4   content                 306496 non-null  object        
 5   score                   306614 non-null  int64         
 6   thumbsUpCount           306614 non-null  int64         
 7   Clean Titles            300230 non-null  object        
 8   neg                     306614 non-null  float64       
 9   neu                     306614 non-null  float64       
 10  pos                     306614 non-null  float64       
 11  compound                306614 non-null  float64       
 12  nb_chars                306614

## Dropping the missing values

### These must be reviews wich only have stopwords which were deleted by the text cleaning formatting. These reviews will anyway not contribute meaning to the sentiment or process analysis. Thus we will drop these reviews.

In [11]:
d.isna().sum()

Activity                     0
Acivity_General              0
TimeStamp                    0
userName                     0
content                    118
score                        0
thumbsUpCount                0
Clean Titles              6384
neg                          0
neu                          0
pos                          0
compound                     0
nb_chars                     0
nb_words                     0
Samsung Gear Mentioned       0
Mi Fit Mentioned             0
Compound Sentiment           0
App                          0
dtype: int64

In [9]:
d.dropna(subset = ['content'], axis = 0, inplace = True)

In [10]:
d.isna().sum()

Activity                     0
Acivity_General              0
TimeStamp                    0
userName                     0
content                      0
score                        0
thumbsUpCount                0
Clean Titles              6266
neg                          0
neu                          0
pos                          0
compound                     0
nb_chars                     0
nb_words                     0
Samsung Gear Mentioned       0
Mi Fit Mentioned             0
Compound Sentiment           0
App                          0
dtype: int64

In [11]:
d['Acivity_General'].value_counts()

5 Star Review        102455
1 Star Review         69888
4 Star Review         38390
Reply from FitBit     37373
3 Star Review         29225
2 Star Review         29165
Name: Acivity_General, dtype: int64

In [13]:
d.shape

(306496, 18)

In [12]:
# return the wordnet object value corresponding to the POS tag
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [14]:
d["Clean Titles"] = d["content"].apply(lambda x: clean_text(x))

In [18]:
d.isna().sum()

Activity                  0
Acivity_General           0
TimeStamp                 0
userName                  0
content                   0
score                     0
thumbsUpCount             0
Clean Titles              0
neg                       0
neu                       0
pos                       0
compound                  0
nb_chars                  0
nb_words                  0
Samsung Gear Mentioned    0
Mi Fit Mentioned          0
Compound Sentiment        0
App                       0
dtype: int64

## Working data for LDA

In [15]:
data = d.copy()

In [16]:
wdata = data['Clean Titles']

## Initiating the LDA model - preparing the data for application

In [17]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=100, # minimum reqd occurences of a word 
                             stop_words='english', # remove stop words
                             lowercase=True, # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}', # num chars > 3
                             # max_features=50000, # max number of uniq words    
                            )
data_vectorized = vectorizer.fit_transform(wdata)

## Cleaned data ready for Topic Modelling

In [23]:
wdata.head()

0    may do critical update get fitbit time need watch
1                       good app motivate great update
2                          always room.for improvement
3    fix bug please forget can't syce phone fix wor...
4                                   work well good inf
Name: Clean Titles, dtype: object

## Initial generic model with random choice of 10 topics

### Although we can take a statistical machine learning approach using GridSearchCV to find the optimum number of topics, I feel that it will be much beyond the scope of this paper. Thus restricting to meaningful topics below 10.

## Creating a model with 20 topics

In [18]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20, # Number of topics
                                      max_iter=10, # Max learning iterations
                                      learning_method='online',   
                                      random_state=100, # Random state
                                      batch_size=128, # n docs in each learning iter
                                      evaluate_every = -1, # compute perplexity every n iters, default: Don't
                                      n_jobs = -1, # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [32]:
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
14     0.137985 -0.128261       1        1  7.806820
16     0.255346  0.009969       2        1  7.411977
11     0.300536 -0.103173       3        1  7.332232
7      0.210313  0.047811       4        1  7.108900
19    -0.015344  0.064911       5        1  6.932455
17     0.241475  0.000442       6        1  6.862463
9     -0.085238 -0.137864       7        1  5.009707
4     -0.192459 -0.104132       8        1  4.826608
3     -0.139928 -0.130907       9        1  4.811167
15     0.076751  0.020060      10        1  4.628533
2     -0.163531 -0.150580      11        1  4.464330
1     -0.138540  0.101792      12        1  4.452805
8      0.037270  0.115693      13        1  4.358809
12    -0.063316 -0.223193      14        1  4.337512
6     -0.148233  0.040559      15        1  3.938412
5     -0.074129  0.193099      16        1  3.825428
10    -0.054886  0.152193      17        1  3.717529
0     -0.058677 -0.163316      18        1  3.129003
18    -0.068731  0.178021      19        1  2.524298
13    -0.056663  0.216877      20        1  2.521011, topic_info=          Term           Freq          Total Category  logprob  loglift
591     fitbit  172967.000000  172967.000000  Default  30.0000  30.0000
268        com   53556.000000   53556.000000  Default  29.0000  29.0000
1562      sync  111604.000000  111604.000000  Default  28.0000  28.0000
710       help   57739.000000   57739.000000  Default  27.0000  27.0000
1549   support   52829.000000   52829.000000  Default  26.0000  26.0000
...        ...            ...            ...      ...      ...      ...
1250     range     241.662347     242.517036  Topic20  -5.8568   3.6770
1281   rectify     232.310671     233.165361  Topic20  -5.8963   3.6768
1779   workout    2069.290555    3127.009626  Topic20  -3.7094   3.2676
25    activity    3766.360640    6905.591264  Topic20  -3.1105   3.0743
85         app   17068.751208  121652.226448  Topic20  -1.5993   1.7166

[798 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0        12  0.999302      ability
1        13  0.999842         able
3        10  0.999725   absolutely
5        13  0.999743       access
9        20  0.999463  accordingly
...     ...       ...          ...
1804      6  0.998724          yes
1805      2  0.797613    yesterday
1805      8  0.201635    yesterday
1809      2  0.999099          zip
1810      8  0.999003         zone

[840 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 17, 12, 8, 20, 18, 10, 5, 4, 16, 3, 2, 9, 13, 7, 6, 11, 1, 19, 14])

In [28]:
ee.sort_values(by = 'Topic16', ascending = False).head(20)

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19
update,0.050000,0.050000,0.05,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.052253,0.050000,0.050070,0.050000,48906.996914,0.050000,0.050000,0.050000
sync,0.050000,0.050000,0.05,0.050000,0.050000,0.050000,0.050000,14962.020655,0.050000,0.050000,0.050000,68913.902888,2.756210,0.050000,0.050000,0.050000,27332.728306,20033.203478,0.050000,0.050000
app,0.050009,6711.813856,0.05,0.050000,0.996768,7067.609883,2094.430429,12358.720266,9418.849198,207.947247,7313.396279,3075.586720,0.050000,16033.861911,0.050000,11501.359213,17853.097170,17421.150504,7457.318918,10776.925232
troubleshoot,0.050000,0.050000,0.05,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,17252.932246,0.050000,0.050000,0.050000
fix,0.050000,0.050000,0.05,0.050000,0.050000,0.050000,0.050000,6481.053918,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,15399.379841,8987.381749,0.050000,60.734781
work,3786.626198,0.050000,0.05,0.050030,0.050000,0.050000,0.050000,0.050000,3408.041560,0.050000,0.050000,2731.826913,24369.798845,0.050000,0.225900,0.050000,11079.271409,7662.030778,0.050000,5206.213231
fitbit,0.914802,0.050000,0.05,16.190543,63.076758,0.050000,0.050000,5622.382608,6595.433545,15.586200,0.050000,22277.283555,0.050000,0.050000,162195.895282,22883.707673,10113.569005,18632.085283,0.050000,188.043092
cause,0.050000,0.050000,0.05,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,8818.363001,0.050000,0.050000,0.050139
battery,0.050000,0.050000,0.05,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,8016.715038,0.050000,0.050000,0.050000
issue,0.050001,0.050000,0.05,0.050000,0.050000,0.050000,0.050000,14129.126430,0.050000,2943.763926,0.050000,29007.385576,0.050000,0.050000,0.050000,0.050000,7299.247939,0.050000,0.050000,0.050000


## Topic for each review

In [20]:
# Create Document — Topic Matrix
# lda_output = lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
# def color_green(val):
#     color = 'green' if val > .1 else 'black'
#     return 'color: {col}'.format(col=color)

# def make_bold(val):
#     weight = 700 if val > .1 else 400
#     return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
# df_document_topics = df_document_topic.style.applymap(color_green).applymap(make_bold)
df_document_topic.head()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.18,...,0.01,0.01,0.01,0.14,0.01,0.26,0.01,0.01,0.01,10
Doc1,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.27,0.01,0.19,0.20,0.01,0.20,0.01,13
Doc2,0.02,0.02,0.02,0.02,0.68,0.02,0.02,0.02,0.02,0.02,...,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,4
Doc3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.28,0.59,0.00,0.07,17
Doc4,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.02,0.35,0.02,0.02,0.02,0.02,0.02,0.35,0.02,12


## Words distribution for each topic

In [21]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords

,ability,able,absolute,absolutely,accept,access,accessible,accomplish,accord,accordingly,...,yay,yeah,year,yes,yesterday,yoga,young,zero,zip,zone
Topic0,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.050000,0.050000,0.050000,0.05000
Topic1,1299.723128,0.050000,0.050000,0.050000,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.050000,0.050000,0.050000,0.05000
Topic2,0.050000,0.050000,0.050000,0.050000,488.892794,0.050000,0.0500,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.050000,0.050000,0.050000,0.05000
Topic3,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.0500,133.342338,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.050000,0.050000,0.050000,0.05000
Topic4,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,386.057644,134.45777,0.050000,0.050000,0.050000,931.96814
Topic5,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,105.5855,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.050000,0.050000,0.050000,0.05000
Topic6,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,89.779736,0.050000,0.050000,0.05000
Topic7,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.0500,0.050000,191.422129,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.050000,0.050000,0.050000,0.05000
Topic8,0.050000,8350.443599,0.050000,0.050000,0.050000,2238.527655,0.0500,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.050000,0.050000,0.050000,0.05000
Topic9,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,...,0.050000,152.734899,0.050000,0.050000,0.050000,0.05000,0.050000,701.848883,0.050000,0.05000


## Transposing the Topic-Word table to correlate the topics in the pyLDAvis visualization and actual data to prevent any confusion during interpretation across different analyses.

In [22]:
ee = df_topic_keywords.T

## Renaming Topics in accordance with the pyLDAvis visualization

In [45]:
ee.sort_values(by = 'Topic1', ascending = False).head(20)

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19
track,0.050000,26608.762467,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.05,0.050000,0.05,0.050000,0.05000,0.050000,0.050000,0.050000
sleep,0.050000,14062.816689,0.050000,0.050000,6750.664677,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.05,0.050000,0.05,0.050000,0.05000,0.050000,0.050000,0.050000
like,0.050000,13319.818951,0.050000,0.050000,0.050000,0.050000,4387.201880,0.050000,0.050000,0.050000,0.050000,0.18676,0.05,0.050000,0.05,0.050000,0.05000,2675.258425,0.050000,5048.903862
additional,0.050000,8618.651292,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.05,0.050000,0.05,0.050000,0.05000,0.050000,0.050000,0.050000
feedback,0.050000,7182.976570,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.05,0.050000,0.05,0.050000,0.05000,0.050000,0.050000,0.050000
app,0.050009,6711.813856,0.050000,0.050000,0.996768,7067.609883,2094.430429,12358.720266,9418.849198,207.947247,7313.396279,3075.58672,0.05,16033.861911,0.05,11501.359213,17853.09717,17421.150504,7457.318918,10776.925232
fitness,0.050000,6130.847258,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.05,0.050000,0.05,0.050000,0.05000,0.050000,0.050000,0.050000
way,0.050000,5849.319234,0.050000,0.050001,0.050000,1498.123282,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.05,0.050000,0.05,0.050000,0.05000,0.050000,0.050000,0.050000
awesome,0.050000,4792.886359,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.05,0.050000,0.05,0.050000,0.05000,0.050000,0.050000,0.050000
info,0.050000,4499.257815,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.05000,0.05,0.050000,0.05,0.050000,0.05000,0.050000,0.050000,0.050000


## Replacing the topic names in accordance with the pyLDAvis visualization.

In [47]:
df_document_topic.tail()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc306491,0.23,0.16,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.08,0.00,0.16,0.08,0.23,0.00,0.00,0.00,0
Doc306492,0.01,0.01,0.28,0.01,0.25,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.16,0.01,0.01,0.11,0.01,0.01,0.13,2
Doc306493,0.00,0.00,0.23,0.00,0.05,0.00,0.10,0.38,0.00,0.10,...,0.00,0.00,0.00,0.00,0.00,0.00,0.12,0.00,0.00,7
Doc306494,0.00,0.00,0.26,0.00,0.00,0.00,0.12,0.30,0.00,0.11,...,0.00,0.00,0.00,0.00,0.00,0.00,0.17,0.00,0.00,7
Doc306495,0.00,0.00,0.26,0.00,0.00,0.00,0.12,0.30,0.00,0.11,...,0.00,0.00,0.00,0.00,0.00,0.00,0.17,0.00,0.00,7


## Distribution of topics

In [48]:
df_document_topic.dominant_topic.value_counts()

16    30201
14    25246
11    24434
1     23603
15    22320
7     21202
17    18741
4     17150
19    16056
5     13814
0     11813
13    11294
8     11141
18    10220
6      9646
10     8983
2      8730
3      8345
12     8333
9      5224
Name: dominant_topic, dtype: int64

## Merging the topics with respective reviews

In [49]:
ed = df_document_topic['dominant_topic'].reset_index()

In [50]:
rr = d.reset_index()

In [51]:
d3 = pd.concat([rr, ed], axis = 1)

In [52]:
d3.drop('index', inplace = True, axis = 1)

## Final dataset with topics for each review

In [53]:
d3.head()

,Activity,Acivity_General,TimeStamp,userName,content,score,thumbsUpCount,Clean Titles,neg,neu,pos,compound,nb_chars,nb_words,Samsung Gear Mentioned,Mi Fit Mentioned,Compound Sentiment,App,dominant_topic
0,2 Star Review for Fit Bit,2 Star Review,2021-03-05 00:16:09,Joseph Kelly,"They may have done a critical update, now all ...",2,0,may do critical update get fitbit time need watch,0.223,0.777,0.000,-0.3182,102,23,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Negative,FitBit,10
1,5 Star Review for Fit Bit,5 Star Review,2021-03-05 00:12:25,Ken Bradford,Good app. Very motivating. Great update.,5,0,good app motivate great update,0.000,0.172,0.828,0.8625,40,6,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Positive,FitBit,13
2,3 Star Review for Fit Bit,3 Star Review,2021-03-05 00:07:48,248baumer X,Always room.for improvement,3,0,always room.for improvement,0.000,0.400,0.600,0.4588,27,3,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Positive,FitBit,4
3,1 Star Review for Fit Bit,1 Star Review,2021-03-05 00:04:48,danny weil,Fix the bug please and don't forget and I can'...,1,3,fix bug please forget can't syce phone fix wor...,0.084,0.711,0.204,0.4019,201,43,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Positive,FitBit,17
4,4 Star Review for Fit Bit,4 Star Review,2021-03-05 00:02:57,Elliott Dudnik,Works well good inf,4,0,work well good inf,0.000,0.286,0.714,0.6124,19,4,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,Positive,FitBit,12


In [55]:
d3.to_excel('FitBit Reviews with Topics.xlsx')

## Removing the anonymous 'A Google user' for analysis in Celonis as the userName is the CaseID. But because I will have to maintain 2 files of such a big size, I will keep it as a filter of exclusion in Celonis.

In [160]:
# d3.userName.value_counts()

A Google user        19065
Adam                    28
Alex                    27
Mike                    25
Mark                    25
                     ...  
Liyah Jenkins            1
Dieter Batz              1
Girish Phadke            1
Migle Z                  1
Stephanie Sharkey        1
Name: userName, Length: 235653, dtype: int64

In [161]:
# d3 = d3[d3['userName']!='A Google user']

In [162]:
# d3.userName.value_counts()

Adam                 28
Alex                 27
Mark                 25
Mike                 25
J S                  25
                     ..
James Daly            1
Liyah Jenkins         1
Dieter Batz           1
Girish Phadke         1
Stephanie Sharkey     1
Name: userName, Length: 235652, dtype: int64

# Mi Fit

## Reading the datasets.

In [56]:
d = pd.read_excel('Xiaomi Reviews.xlsx', parse_dates = True)

## Keeping a copy of the original dataset, just in case.

In [57]:
d1 = d.copy()

In [58]:
d.head()

,Unnamed: 0,Activity,Activity_Genral,TimeStamp,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,...,neg,neu,pos,compound,nb_chars,nb_words,FitBit Mentioned,Samsung Gear Mentioned,Compound Sentiment,App
0,0,5 Star Review for Xiaomi,5 Star Review,2021-03-04 23:40:57,gp:AOqpTOFXA4ln-LqPrXCf_o23feuyvR4Er7ilH96bBj6...,Vijaya Kulkarni,Its not getting updated,5,0,4.8.1,...,0.0,1.0,0.0,0.0000,23,4,FitBit Not Mentioned,Samsung Gear Not Mentioned,Neutral,Mi Fit
1,1,5 Star Review for Xiaomi,5 Star Review,2021-03-04 22:56:22,gp:AOqpTOEOo1y73TRoUfoogGjHqOmyxS3xJUSD0rYB-D0...,Srujana Awanti,Good,5,0,NaN,...,0.0,0.0,1.0,0.4404,4,1,FitBit Not Mentioned,Samsung Gear Not Mentioned,Positive,Mi Fit
2,2,4 Star Review for Xiaomi,4 Star Review,2021-03-04 22:39:48,gp:AOqpTOF00Jzsj8U5u74dnpkxlvoFkSh1_Pu6oO-tNIU...,worship song Kalbo,Good,4,0,4.9.0,...,0.0,0.0,1.0,0.4404,4,1,FitBit Not Mentioned,Samsung Gear Not Mentioned,Positive,Mi Fit
3,3,5 Star Review for Xiaomi,5 Star Review,2021-03-04 22:26:59,gp:AOqpTOFEeZ761ulmPDwSxAiVK930v_VL3EO5BdWy2OT...,suresh b,Fun,5,0,4.3.1,...,0.0,0.0,1.0,0.5106,3,1,FitBit Not Mentioned,Samsung Gear Not Mentioned,Positive,Mi Fit
4,4,5 Star Review for Xiaomi,5 Star Review,2021-03-04 22:25:50,gp:AOqpTOFL8f4YxZ1oY9w2nvHgpMQNCPi4pWTqCQbBQsZ...,Jawaid Iqbal,Jk,5,0,4.9.0,...,0.0,0.0,1.0,0.2263,2,1,FitBit Not Mentioned,Samsung Gear Not Mentioned,Positive,Mi Fit


## Dropping the identifier columns

In [59]:
d.drop(['Unnamed: 0', 'reviewId', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt'], axis = 1, inplace = True)

In [60]:
d.head()

,Activity,Activity_Genral,TimeStamp,userName,content,score,thumbsUpCount,Clean Titles,neg,neu,pos,compound,nb_chars,nb_words,FitBit Mentioned,Samsung Gear Mentioned,Compound Sentiment,App
0,5 Star Review for Xiaomi,5 Star Review,2021-03-04 23:40:57,Vijaya Kulkarni,Its not getting updated,5,0,get update,0.0,1.0,0.0,0.0000,23,4,FitBit Not Mentioned,Samsung Gear Not Mentioned,Neutral,Mi Fit
1,5 Star Review for Xiaomi,5 Star Review,2021-03-04 22:56:22,Srujana Awanti,Good,5,0,good,0.0,0.0,1.0,0.4404,4,1,FitBit Not Mentioned,Samsung Gear Not Mentioned,Positive,Mi Fit
2,4 Star Review for Xiaomi,4 Star Review,2021-03-04 22:39:48,worship song Kalbo,Good,4,0,good,0.0,0.0,1.0,0.4404,4,1,FitBit Not Mentioned,Samsung Gear Not Mentioned,Positive,Mi Fit
3,5 Star Review for Xiaomi,5 Star Review,2021-03-04 22:26:59,suresh b,Fun,5,0,fun,0.0,0.0,1.0,0.5106,3,1,FitBit Not Mentioned,Samsung Gear Not Mentioned,Positive,Mi Fit
4,5 Star Review for Xiaomi,5 Star Review,2021-03-04 22:25:50,Jawaid Iqbal,Jk,5,0,jk,0.0,0.0,1.0,0.2263,2,1,FitBit Not Mentioned,Samsung Gear Not Mentioned,Positive,Mi Fit


## Shape of the data

In [61]:
d.shape

(129635, 18)

## Basic description of the dataset

In [62]:
d.describe(include = 'all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
Activity,129635,6,5 Star Review for Xiaomi,58036,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Activity_Genral,129635,6,5 Star Review,58036,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TimeStamp,129635,129537,2020-02-19 18:50:40,2,2014-10-24 19:29:19,2021-03-04 23:40:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN
userName,129635,117963,A Google user,6162,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
content,129629,95380,Good,5570,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
score,129635,NaN,NaN,NaN,NaT,NaT,3.4986,1.64162,0,2,4,5,5
thumbsUpCount,129635,NaN,NaN,NaN,NaT,NaT,0.90582,13.0546,0,0,0,0,1208
Clean Titles,128818,85179,good,9404,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
neg,129635,NaN,NaN,NaN,NaT,NaT,0.0608659,0.155658,0,0,0,0,1
neu,129635,NaN,NaN,NaN,NaT,NaT,0.516402,0.363101,0,0.213,0.537,0.84,1


In [63]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129635 entries, 0 to 129634
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Activity                129635 non-null  object        
 1   Activity_Genral         129635 non-null  object        
 2   TimeStamp               129635 non-null  datetime64[ns]
 3   userName                129635 non-null  object        
 4   content                 129629 non-null  object        
 5   score                   129635 non-null  int64         
 6   thumbsUpCount           129635 non-null  int64         
 7   Clean Titles            128818 non-null  object        
 8   neg                     129635 non-null  float64       
 9   neu                     129635 non-null  float64       
 10  pos                     129635 non-null  float64       
 11  compound                129635 non-null  float64       
 12  nb_chars                129635

## Dropping the missing values

### These must be reviews wich only have stopwords which were deleted by the text cleaning formatting. These reviews will anyway not contribute meaning to the sentiment or process analysis. Thus we will drop these reviews.

In [64]:
d.isna().sum()

Activity                    0
Activity_Genral             0
TimeStamp                   0
userName                    0
content                     6
score                       0
thumbsUpCount               0
Clean Titles              817
neg                         0
neu                         0
pos                         0
compound                    0
nb_chars                    0
nb_words                    0
FitBit Mentioned            0
Samsung Gear Mentioned      0
Compound Sentiment          0
App                         0
dtype: int64

In [65]:
d.dropna(subset = ['content'], axis = 0, inplace = True)

In [66]:
d.isna().sum()

Activity                    0
Activity_Genral             0
TimeStamp                   0
userName                    0
content                     0
score                       0
thumbsUpCount               0
Clean Titles              811
neg                         0
neu                         0
pos                         0
compound                    0
nb_chars                    0
nb_words                    0
FitBit Mentioned            0
Samsung Gear Mentioned      0
Compound Sentiment          0
App                         0
dtype: int64

In [67]:
d["Clean Titles"] = d["content"].apply(lambda x: clean_text(x))

In [68]:
d.Activity.value_counts()

5 Star Review for Xiaomi        58033
1 Star Review for Xiaomi        30298
4 Star Review for Xiaomi        18633
3 Star Review for Xiaomi        13236
2 Star Review for Xiaomi         9407
Reply from Xiaomi for Xiaomi       22
Name: Activity, dtype: int64

In [69]:
d.shape

(129629, 18)

## Working data for LDA

In [70]:
data = d.copy()

In [71]:
wdata = data['Clean Titles']

## Initiating the LDA model - preparing the data for application

In [72]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=100, # minimum reqd occurences of a word 
                             stop_words='english', # remove stop words
                             lowercase=True, # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}', # num chars > 3
                             # max_features=50000, # max number of uniq words    
                            )
data_vectorized = vectorizer.fit_transform(wdata)

## Cleaned data ready for Topic Modelling

In [73]:
wdata.head()

0    get update
1          good
2          good
3           fun
4            jk
Name: Clean Titles, dtype: object

## Initial generic model with random choice of 10 topics

### Although we can take a statistical machine learning approach using GridSearchCV to find the optimum number of topics, I feel that it will be much beyond the scope of this paper. Thus restricting to meaningful topics below 10.

In [74]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20, # Number of topics
                                      max_iter=10, # Max learning iterations
                                      learning_method='online',   
                                      random_state=100, # Random state
                                      batch_size=128, # n docs in each learning iter
                                      evaluate_every = -1, # compute perplexity every n iters, default: Don't
                                      n_jobs = -1, # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


## General visualization of the topics.

In [75]:
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.148957  0.290384       1        1  9.450434
2     -0.054138 -0.113270       2        1  8.252076
0     -0.111345  0.148703       3        1  7.359280
1     -0.134698  0.257380       4        1  6.677741
3      0.385992  0.085213       5        1  5.899552
16    -0.042182 -0.076271       6        1  5.805720
5     -0.058460 -0.077208       7        1  5.148333
15    -0.116707  0.228291       8        1  5.134045
19    -0.039360 -0.088065       9        1  4.376389
7     -0.040865 -0.083646      10        1  4.224219
18     0.340471  0.048532      11        1  4.185134
12    -0.055116 -0.118495      12        1  4.078283
11    -0.047610 -0.061561      13        1  4.033442
6      0.032105 -0.093607      14        1  4.001586
8     -0.039258 -0.086422      15        1  3.981890
4     -0.044848 -0.114887      16        1  3.855039
10    -0.048473 -0.049785      17        1  3.821238
17    -0.048468 -0.110245      18        1  3.361706
14     0.334007  0.073546      19        1  3.192357
9     -0.062090 -0.058588      20        1  3.161538, topic_info=        Term          Freq         Total Category  logprob  loglift
36       app  41572.000000  41572.000000  Default  30.0000  30.0000
278     good  20828.000000  20828.000000  Default  29.0000  29.0000
709   update  14996.000000  14996.000000  Default  28.0000  28.0000
752     work  14132.000000  14132.000000  Default  27.0000  27.0000
58      band  27480.000000  27480.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
214    error      0.049737   1304.743695  Topic20 -12.9212  -6.7207
45   attempt      0.049737    187.219830  Topic20 -12.9212  -4.7792
275    gmail      0.049737    139.845316  Topic20 -12.9212  -4.4874
75     blank      0.049737    133.138748  Topic20 -12.9212  -4.4383
350    later      0.049737    243.269138  Topic20 -12.9212  -5.0411

[725 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0        15  0.997396         aap
1         8  0.997206     ability
2         1  0.999823        able
3        19  0.996825  absolutely
5        10  0.997966      access
...     ...       ...         ...
764       4  0.992316       xiomi
765       4  0.997814        year
766      19  0.997282         yes
767       2  0.629703   yesterday
767       3  0.370031   yesterday

[624 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 3, 1, 2, 4, 17, 6, 16, 20, 8, 19, 13, 12, 7, 9, 5, 11, 18, 15, 10])

## Topic for each review

In [76]:
# Create Document — Topic Matrix
# lda_output = lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
# def color_green(val):
#     color = 'green' if val > .1 else 'black'
#     return 'color: {col}'.format(col=color)

# def make_bold(val):
#     weight = 700 if val > .1 else 400
#     return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
# df_document_topics = df_document_topic.style.applymap(color_green).applymap(make_bold)
df_document_topic.head()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.52,0.03,0.03,0.03,16
Doc1,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.52,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,8
Doc2,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.52,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,8
Doc3,0.03,0.52,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,1
Doc4,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0


## Words distribution for each topic

In [77]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords

,aap,ability,able,absolutely,accept,access,account,accuracy,accurate,accurately,...,wow,wrist,write,wrong,xiaomi,xiomi,year,yes,yesterday,zero
Topic0,0.050000,0.050000,0.050000,0.050000,49.259925,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,161.701094,0.050000,0.050000,0.050000,0.050000,0.050000,127.785912,176.803925
Topic1,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,131.306969,656.845616,0.050000,0.050000,0.050000
Topic2,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,1890.535727,194.856849,...,0.050000,0.050000,0.050000,890.177673,0.050000,0.050000,0.050000,0.050000,220.208883,0.050000
Topic3,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
Topic4,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
Topic5,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,437.472756,0.050000,0.050000,0.050000,1971.405703,0.050000,0.050000,0.050000,0.050000,0.050000
Topic6,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,330.434417,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
Topic7,0.050000,0.050000,0.050000,0.050000,0.050000,603.977723,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,37.452529,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
Topic8,409.565797,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
Topic9,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,3117.989253,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000


## Transposing the Topic-Word table to correlate the topics in the pyLDAvis visualization and actual data to prevent any confusion during interpretation across different analyses.

In [78]:
ee = df_topic_keywords.T

In [79]:
ee.sort_values(by = 'Topic5', ascending = False).head(20)

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19
like,0.05,0.05,0.05,0.05,0.05,4912.844629,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,979.869397,0.05,0.05
support,0.05,0.05,0.05,0.05,0.05,2071.526613,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.050000,0.05,0.05
xiaomi,0.05,0.05,0.05,0.05,0.05,1971.405703,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.050000,0.05,0.05
lot,0.05,0.05,0.05,0.05,0.05,1590.264168,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.050000,0.05,0.05
turn,0.05,0.05,0.05,0.05,0.05,1269.835545,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.050000,0.05,0.05
month,0.05,0.05,0.05,0.05,0.05,1165.949719,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.050000,0.05,0.05
download,0.05,0.05,0.05,0.05,0.05,1160.561283,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.050000,0.05,0.05
information,0.05,0.05,0.05,0.05,0.05,948.801292,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.050000,0.05,0.05
music,0.05,0.05,0.05,0.05,0.05,945.702676,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.050000,0.05,0.05
scale,0.05,0.05,0.05,0.05,0.05,876.235498,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.050000,0.05,0.05,0.05,0.050006,0.05,0.05


## Renaming Topics in accordance with the pyLDAvis visualization

## Replacing the topic names in accordance with the pyLDAvis visualization.

In [80]:
df_document_topic.head()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.52,0.03,0.03,0.03,16
Doc1,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.52,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,8
Doc2,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.52,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,8
Doc3,0.03,0.52,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,1
Doc4,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0


## Distribution of topics

In [81]:
df_document_topic.dominant_topic.value_counts()

0     18943
8     13335
13    12237
3     12104
2     10501
1      9574
18     8321
5      4878
11     4326
16     4313
15     4269
14     4038
9      3945
4      3695
7      3477
6      3032
19     2903
12     2331
10     2115
17     1292
Name: dominant_topic, dtype: int64

## Merging the topics with respective reviews

In [82]:
ed = df_document_topic['dominant_topic'].reset_index()

In [83]:
rr = d.reset_index()

In [84]:
d4 = pd.concat([rr, ed], axis = 1)

In [85]:
d4.drop('index', inplace = True, axis = 1)

## Final dataset with topics for each review

In [86]:
d4.head()

,Activity,Activity_Genral,TimeStamp,userName,content,score,thumbsUpCount,Clean Titles,neg,neu,pos,compound,nb_chars,nb_words,FitBit Mentioned,Samsung Gear Mentioned,Compound Sentiment,App,dominant_topic
0,5 Star Review for Xiaomi,5 Star Review,2021-03-04 23:40:57,Vijaya Kulkarni,Its not getting updated,5,0,get update,0.0,1.0,0.0,0.0000,23,4,FitBit Not Mentioned,Samsung Gear Not Mentioned,Neutral,Mi Fit,16
1,5 Star Review for Xiaomi,5 Star Review,2021-03-04 22:56:22,Srujana Awanti,Good,5,0,good,0.0,0.0,1.0,0.4404,4,1,FitBit Not Mentioned,Samsung Gear Not Mentioned,Positive,Mi Fit,8
2,4 Star Review for Xiaomi,4 Star Review,2021-03-04 22:39:48,worship song Kalbo,Good,4,0,good,0.0,0.0,1.0,0.4404,4,1,FitBit Not Mentioned,Samsung Gear Not Mentioned,Positive,Mi Fit,8
3,5 Star Review for Xiaomi,5 Star Review,2021-03-04 22:26:59,suresh b,Fun,5,0,fun,0.0,0.0,1.0,0.5106,3,1,FitBit Not Mentioned,Samsung Gear Not Mentioned,Positive,Mi Fit,1
4,5 Star Review for Xiaomi,5 Star Review,2021-03-04 22:25:50,Jawaid Iqbal,Jk,5,0,jk,0.0,0.0,1.0,0.2263,2,1,FitBit Not Mentioned,Samsung Gear Not Mentioned,Positive,Mi Fit,0


In [87]:
d4.to_excel('Xiaomi Reviews with Topics.xlsx')

# Samsung

## Reading the datasets.

In [88]:
d = pd.read_excel('Samsung Reviews.xlsx', parse_dates = True)

## Keeping a copy of the original dataset, just in case.

In [89]:
d1 = d.copy()

In [90]:
d.head()

,Unnamed: 0,Activity,Activity_General,TimeStamp,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,...,neg,neu,pos,compound,nb_chars,nb_words,Mi Fit Mentioned,FitBit Mentioned,Compound Sentiment,App
0,0,1 Star Review for Samsung,1 Star Review,2021-03-04 23:58:56,gp:AOqpTOFcNvos4kr_pvPFnI6y2QG2xWblNaZfRc-hknm...,hamid kainouch,The app suddenly stopped couting steps!!!,1,0,6.15.5.019,...,0.355,0.645,0.000,-0.2960,41,6,Mi Fit Not Mentioned,FitBit Not Mentioned,Negative,Samsung
1,1,2 Star Review for Samsung,2 Star Review,2021-03-04 23:12:15,gp:AOqpTOFVzDAoKdTd_OIchHP3w5nuqC1yJ7nxmma3PpP...,Taylor Hohl,"No longer able to manually enter workouts, unl...",2,0,6.15.5.019,...,0.000,1.000,0.000,0.0000,157,30,Mi Fit Not Mentioned,FitBit Not Mentioned,Neutral,Samsung
2,2,1 Star Review for Samsung,1 Star Review,2021-03-04 23:10:19,gp:AOqpTOETgaLEWdM5rA8vK2WuqIIqM0i36cSHYtAtahq...,Jason Vaughan,This app took a dump and no longer works prope...,1,0,6.15.5.019,...,0.355,0.645,0.000,-0.6705,124,22,Mi Fit Not Mentioned,FitBit Not Mentioned,Negative,Samsung
3,3,2 Star Review for Samsung,2 Star Review,2021-03-04 23:01:45,gp:AOqpTOER7l2Aq0acAnPTV00XAsAHJiEfSynqsgfT62Q...,Morgan Evans,"I loved this app before, but getting rid of th...",2,0,6.15.5.019,...,0.092,0.631,0.277,0.8958,471,88,Mi Fit Not Mentioned,FitBit Not Mentioned,Positive,Samsung
4,4,2 Star Review for Samsung,2 Star Review,2021-03-04 22:33:35,gp:AOqpTOEVqgMazHkFLqf1tmkGgcQG-B19hNXUrZkyl_C...,Minoo Norouzi,It doesn't count steps for four deays,2,0,6.15.5.019,...,0.000,1.000,0.000,0.0000,37,7,Mi Fit Not Mentioned,FitBit Not Mentioned,Neutral,Samsung


## Dropping the identifier columns

In [91]:
d.drop(['Unnamed: 0', 'reviewId', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt'], axis = 1, inplace = True)

In [92]:
d.head()

,Activity,Activity_General,TimeStamp,userName,content,score,thumbsUpCount,Clean Titles,neg,neu,pos,compound,nb_chars,nb_words,Mi Fit Mentioned,FitBit Mentioned,Compound Sentiment,App
0,1 Star Review for Samsung,1 Star Review,2021-03-04 23:58:56,hamid kainouch,The app suddenly stopped couting steps!!!,1,0,app suddenly stop couting step,0.355,0.645,0.000,-0.2960,41,6,Mi Fit Not Mentioned,FitBit Not Mentioned,Negative,Samsung
1,2 Star Review for Samsung,2 Star Review,2021-03-04 23:12:15,Taylor Hohl,"No longer able to manually enter workouts, unl...",2,0,longer able manually enter workout unless work...,0.000,1.000,0.000,0.0000,157,30,Mi Fit Not Mentioned,FitBit Not Mentioned,Neutral,Samsung
2,1 Star Review for Samsung,1 Star Review,2021-03-04 23:10:19,Jason Vaughan,This app took a dump and no longer works prope...,1,0,app take dump longer work properly frustrating...,0.355,0.645,0.000,-0.6705,124,22,Mi Fit Not Mentioned,FitBit Not Mentioned,Negative,Samsung
3,2 Star Review for Samsung,2 Star Review,2021-03-04 23:01:45,Morgan Evans,"I loved this app before, but getting rid of th...",2,0,love app get rid weight management feature mas...,0.092,0.631,0.277,0.8958,471,88,Mi Fit Not Mentioned,FitBit Not Mentioned,Positive,Samsung
4,2 Star Review for Samsung,2 Star Review,2021-03-04 22:33:35,Minoo Norouzi,It doesn't count steps for four deays,2,0,count step four deays,0.000,1.000,0.000,0.0000,37,7,Mi Fit Not Mentioned,FitBit Not Mentioned,Neutral,Samsung


## Shape of the data

In [93]:
d.shape

(268861, 18)

## Basic description of the dataset

In [94]:
d.describe(include = 'all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
Activity,268861,6,Reply from Samsung for Samsung,125426,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Activity_General,268861,6,Reply from Samsung,125426,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TimeStamp,268861,268532,2017-04-20 07:52:19,6,2015-04-08 19:05:23,2021-03-04 23:58:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN
userName,268861,128731,A Google user,19354,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
content,268852,230151,Good,3126,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
score,268861,NaN,NaN,NaN,NaT,NaT,3.64707,1.5732,1,2,4,5,5
thumbsUpCount,268861,NaN,NaN,NaN,NaT,NaT,0.695642,7.14002,0,0,0,0,683
Clean Titles,142604,103689,good,6174,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
neg,268861,NaN,NaN,NaN,NaT,NaT,0.0295454,0.0977626,0,0,0,0,1
neu,268861,NaN,NaN,NaN,NaT,NaT,0.273033,0.35261,0,0,0,0.574,1


In [95]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268861 entries, 0 to 268860
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Activity            268861 non-null  object        
 1   Activity_General    268861 non-null  object        
 2   TimeStamp           268861 non-null  datetime64[ns]
 3   userName            268861 non-null  object        
 4   content             268852 non-null  object        
 5   score               268861 non-null  int64         
 6   thumbsUpCount       268861 non-null  int64         
 7   Clean Titles        142604 non-null  object        
 8   neg                 268861 non-null  float64       
 9   neu                 268861 non-null  float64       
 10  pos                 268861 non-null  float64       
 11  compound            268861 non-null  float64       
 12  nb_chars            268861 non-null  int64         
 13  nb_words            268861 no

## Dropping the missing values

### These must be reviews wich only have stopwords which were deleted by the text cleaning formatting. These reviews will anyway not contribute meaning to the sentiment or process analysis. Thus we will drop these reviews.

In [96]:
d.isna().sum()

Activity                   0
Activity_General           0
TimeStamp                  0
userName                   0
content                    9
score                      0
thumbsUpCount              0
Clean Titles          126257
neg                        0
neu                        0
pos                        0
compound                   0
nb_chars                   0
nb_words                   0
Mi Fit Mentioned           0
FitBit Mentioned           0
Compound Sentiment         0
App                        0
dtype: int64

In [97]:
d.dropna(subset = ['content'], axis = 0, inplace = True)

In [98]:
d.isna().sum()

Activity                   0
Activity_General           0
TimeStamp                  0
userName                   0
content                    0
score                      0
thumbsUpCount              0
Clean Titles          126248
neg                        0
neu                        0
pos                        0
compound                   0
nb_chars                   0
nb_words                   0
Mi Fit Mentioned           0
FitBit Mentioned           0
Compound Sentiment         0
App                        0
dtype: int64

In [99]:
d["Clean Titles"] = d["content"].apply(lambda x: clean_text(x))

In [100]:
d.shape

(268852, 18)

## Working data for LDA

In [101]:
data = d.copy()

In [102]:
wdata = data['Clean Titles']

## Initiating the LDA model - preparing the data for application

In [103]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=100, # minimum reqd occurences of a word 
                             stop_words='english', # remove stop words
                             lowercase=True, # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}', # num chars > 3
                             # max_features=50000, # max number of uniq words    
                            )
data_vectorized = vectorizer.fit_transform(wdata)

## Cleaned data ready for Topic Modelling

In [104]:
wdata.head()

0                       app suddenly stop couting step
1    longer able manually enter workout unless work...
2    app take dump longer work properly frustrating...
3    love app get rid weight management feature mas...
4                                count step four deays
Name: Clean Titles, dtype: object

## Initial generic model with random choice of 10 topics

### Although we can take a statistical machine learning approach using GridSearchCV to find the optimum number of topics, I feel that it will be much beyond the scope of this paper. Thus restricting to meaningful topics below 10.

## Creating another model with 5 topics

In [105]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20, # Number of topics
                                      max_iter=10, # Max learning iterations
                                      learning_method='online',   
                                      random_state=100, # Random state
                                      batch_size=128, # n docs in each learning iter
                                      evaluate_every = -1, # compute perplexity every n iters, default: Don't
                                      n_jobs = -1, # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [106]:
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7      0.133776 -0.026605       1        1  9.569363
16     0.143515 -0.029921       2        1  8.318007
4      0.106187 -0.047598       3        1  6.201883
5     -0.248419 -0.112814       4        1  6.128613
0     -0.106213 -0.143474       5        1  5.708232
10     0.140335  0.030911       6        1  5.339169
19     0.132987 -0.048202       7        1  5.194542
15    -0.229279  0.112532       8        1  5.174682
18     0.126817 -0.021319       9        1  4.967017
8     -0.086303 -0.069710      10        1  4.740715
2     -0.205211  0.128243      11        1  4.562727
6      0.040685  0.351637      12        1  4.470652
13     0.090396  0.063139      13        1  4.375647
17    -0.153832 -0.142663      14        1  4.173832
14     0.124172 -0.031288      15        1  3.986922
3     -0.201747  0.180951      16        1  3.869475
12     0.135674 -0.041335      17        1  3.743056
9      0.132044 -0.016903      18        1  3.419723
11     0.126907 -0.002008      19        1  3.034914
1     -0.202493 -0.133573      20        1  3.020826, topic_info=             Term           Freq          Total Category  logprob  loglift
549        health  126509.000000  126509.000000  Default  30.0000  30.0000
1112      samsung  100671.000000  100671.000000  Default  29.0000  29.0000
1278        thank   89005.000000   89005.000000  Default  28.0000  28.0000
82    application   71080.000000   71080.000000  Default  27.0000  27.0000
639         issue   60454.000000   60454.000000  Default  26.0000  26.0000
...           ...            ...            ...      ...      ...      ...
151         board      97.216937      98.132813  Topic20  -6.7182   3.4903
622   instruction      95.702599      96.618475  Topic20  -6.7339   3.4901
138       benefit      91.413613      92.329489  Topic20  -6.7797   3.4897
422     excercise      75.491227      76.407103  Topic20  -6.9711   3.4876
78            app   12922.057792  123270.511637  Topic20  -1.8284   1.2442

[707 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         9  0.990300         aap
5         7  0.997873     ability
6        11  0.784767        able
6        13  0.214942        able
7         8  0.998294  absolutely
...     ...       ...         ...
1419     14  0.998443       world
1422     15  0.998147         wow
1425     10  0.998166       wrong
1428     12  0.999351        year
1430     15  0.994908   yesterday

[705 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 17, 5, 6, 1, 11, 20, 16, 19, 9, 3, 7, 14, 18, 15, 4, 13, 10, 12, 2])

## Topic for each review

In [107]:
# Create Document — Topic Matrix
# lda_output = lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
# def color_green(val):
#     color = 'green' if val > .1 else 'black'
#     return 'color: {col}'.format(col=color)

# def make_bold(val):
#     weight = 700 if val > .1 else 400
#     return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
# df_document_topics = df_document_topic.style.applymap(color_green).applymap(make_bold)
df_document_topic.head()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0.61,0.01,0.01,0.01,0.01,0.01,0.01,0.21,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0
Doc1,0.09,0.00,0.28,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.50,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11
Doc2,0.01,0.26,0.01,0.32,0.01,0.01,0.01,0.01,0.01,0.01,...,0.12,0.11,0.01,0.01,0.01,0.01,0.12,0.01,0.01,3
Doc3,0.00,0.00,0.08,0.00,0.03,0.00,0.00,0.00,0.21,0.06,...,0.03,0.03,0.00,0.00,0.04,0.04,0.00,0.00,0.33,19
Doc4,0.68,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0


## Words distribution for each topic

In [108]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords

,aap,aaron,abdul,abdullah,abhishek,ability,able,absolutely,accept,access,...,write,wrong,xiaomi,yadav,year,yes,yesterday,yoga,young,zero
Topic0,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,132.102091,0.050000
Topic1,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
Topic2,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,1812.315228,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
Topic3,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
Topic4,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,673.409125,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
Topic5,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,109.322261,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
Topic6,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,92.507514,0.050000,1225.972264,0.050000,0.050000,0.050000,0.050000,0.050000
Topic7,0.050000,158.117345,0.050000,109.626638,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,203.891172,0.050000,0.050000,0.050000,0.050000
Topic8,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,539.621887,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
Topic9,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,...,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000


## Transposing the Topic-Word table to correlate the topics in the pyLDAvis visualization and actual data to prevent any confusion during interpretation across different analyses.

In [109]:
ee = df_topic_keywords.T

In [110]:
ee.sort_values(by = 'Topic5', ascending = False).head(20)

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19
app,11079.046221,12502.842146,7247.117234,6234.901098,2.619700,53671.470518,15.089599,146.10046,4754.842914,0.05,79.981955,0.050003,12.3581,73.556496,108.241959,20139.440656,0.050019,11353.751304,66.126671,20.761529
contact,0.050000,0.050000,0.050000,0.050000,0.050000,47087.533858,0.050000,0.05000,0.050000,0.05,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
know,0.050000,0.050000,0.050000,0.050000,0.050000,24850.215397,0.050000,0.05000,0.050000,0.05,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
hello,0.050000,0.050000,0.050000,0.050000,0.050000,17771.514844,0.050000,0.05000,0.050000,0.05,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
speak,0.050000,0.050000,0.050000,0.050000,0.050000,5809.730827,0.050000,0.05000,0.050000,0.05,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
service,0.050000,0.050000,0.050000,0.050000,0.050000,4250.258798,0.050000,0.05000,0.050000,0.05,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
understand,0.050000,0.050000,0.050000,0.050000,0.050000,3156.675760,0.050000,0.05000,0.050000,0.05,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
open,0.050000,0.050000,0.050000,0.050000,0.050000,2954.235733,0.050000,0.05000,0.050000,0.05,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
phone,0.050000,0.050000,7180.662268,0.050000,0.050023,2695.236997,0.050000,0.05000,0.050000,0.05,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000
release,0.050000,0.050000,0.050000,0.050000,0.050000,2679.672298,0.050000,0.05000,0.050000,0.05,0.050000,0.050000,0.0500,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000


In [111]:
df_document_topic.head()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0.61,0.01,0.01,0.01,0.01,0.01,0.01,0.21,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0
Doc1,0.09,0.00,0.28,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.50,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11
Doc2,0.01,0.26,0.01,0.32,0.01,0.01,0.01,0.01,0.01,0.01,...,0.12,0.11,0.01,0.01,0.01,0.01,0.12,0.01,0.01,3
Doc3,0.00,0.00,0.08,0.00,0.03,0.00,0.00,0.00,0.21,0.06,...,0.03,0.03,0.00,0.00,0.04,0.04,0.00,0.00,0.33,19
Doc4,0.68,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0


## Distribution of topics

In [112]:
df_document_topic.dominant_topic.value_counts()

7     50090
0     26332
15    20388
5     18747
1     15829
2     15234
10    14895
4     14390
16    14208
3     13425
8     11248
19     9117
12     8278
13     8264
17     8100
18     6454
6      6173
11     3381
14     2853
9      1446
Name: dominant_topic, dtype: int64

## Merging the topics with respective reviews

In [113]:
ed = df_document_topic['dominant_topic'].reset_index()

In [114]:
rr = d.reset_index()

In [115]:
d5 = pd.concat([rr, ed], axis = 1)

In [116]:
d5.drop('index', inplace = True, axis = 1)

## Final dataset with topics for each review

In [117]:
d5.head()

,Activity,Activity_General,TimeStamp,userName,content,score,thumbsUpCount,Clean Titles,neg,neu,pos,compound,nb_chars,nb_words,Mi Fit Mentioned,FitBit Mentioned,Compound Sentiment,App,dominant_topic
0,1 Star Review for Samsung,1 Star Review,2021-03-04 23:58:56,hamid kainouch,The app suddenly stopped couting steps!!!,1,0,app suddenly stop couting step,0.355,0.645,0.000,-0.2960,41,6,Mi Fit Not Mentioned,FitBit Not Mentioned,Negative,Samsung,0
1,2 Star Review for Samsung,2 Star Review,2021-03-04 23:12:15,Taylor Hohl,"No longer able to manually enter workouts, unl...",2,0,longer able manually enter workout unless work...,0.000,1.000,0.000,0.0000,157,30,Mi Fit Not Mentioned,FitBit Not Mentioned,Neutral,Samsung,11
2,1 Star Review for Samsung,1 Star Review,2021-03-04 23:10:19,Jason Vaughan,This app took a dump and no longer works prope...,1,0,app take dump longer work properly frustrating...,0.355,0.645,0.000,-0.6705,124,22,Mi Fit Not Mentioned,FitBit Not Mentioned,Negative,Samsung,3
3,2 Star Review for Samsung,2 Star Review,2021-03-04 23:01:45,Morgan Evans,"I loved this app before, but getting rid of th...",2,0,love app get rid weight management feature mas...,0.092,0.631,0.277,0.8958,471,88,Mi Fit Not Mentioned,FitBit Not Mentioned,Positive,Samsung,19
4,2 Star Review for Samsung,2 Star Review,2021-03-04 22:33:35,Minoo Norouzi,It doesn't count steps for four deays,2,0,count step four deays,0.000,1.000,0.000,0.0000,37,7,Mi Fit Not Mentioned,FitBit Not Mentioned,Neutral,Samsung,0


In [118]:
d5.to_excel('Samsung Reviews with Topics.xlsx')

In [125]:
d6 = pd.read_excel('KPMG Data with Topics and Encoding V1 - Final.xlsx', parse_dates = True)

In [126]:
d6.head()

,Unnamed: 0,Activity,Acivity_General,TimeStamp,userName,content,score,thumbsUpCount,Clean Titles,neg,...,pos,compound,nb_chars,nb_words,Samsung Gear Mentioned,Mi Fit Mentioned,FitBit Mentioned,Compound Sentiment,App,dominant_topic
0,0,2 Star Review for Fit Bit,2 Star Review,2021-03-05 00:16:09,Joseph Kelly,"They may have done a critical update, now all ...",2,0,may do critical update get fitbit time need watch,0.223,...,0.000,-0.3182,102,23,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,NaN,Negative,FitBit,10
1,1,5 Star Review for Fit Bit,5 Star Review,2021-03-05 00:12:25,Ken Bradford,Good app. Very motivating. Great update.,5,0,good app motivate great update,0.000,...,0.828,0.8625,40,6,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,NaN,Positive,FitBit,13
2,2,3 Star Review for Fit Bit,3 Star Review,2021-03-05 00:07:48,248baumer X,Always room.for improvement,3,0,always room.for improvement,0.000,...,0.600,0.4588,27,3,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,NaN,Positive,FitBit,4
3,3,1 Star Review for Fit Bit,1 Star Review,2021-03-05 00:04:48,danny weil,Fix the bug please and don't forget and I can'...,1,3,fix bug please forget can't syce phone fix wor...,0.084,...,0.204,0.4019,201,43,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,NaN,Positive,FitBit,17
4,4,4 Star Review for Fit Bit,4 Star Review,2021-03-05 00:02:57,Elliott Dudnik,Works well good inf,4,0,work well good inf,0.000,...,0.714,0.6124,19,4,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,NaN,Positive,FitBit,12


In [128]:
d6.shape

(704977, 21)

In [129]:
d6['userName'].nunique()

467737

In [130]:
d6['userName'].value_counts()

A Google user        44684
Alex                    70
Chris                   57
Amit Kumar              56
Google User             48
                     ...  
Noa Greén                1
Lilly Montes             1
thushan madusanka        1
Rp Bautista              1
James Kern               1
Name: userName, Length: 467737, dtype: int64

In [131]:
d7 = d6[d6['userName'] != 'A Google user']

In [132]:
d7['userName'].nunique()

467736

In [134]:
d7['userName'].value_counts()

Alex                 70
Chris                57
Amit Kumar           56
Google User          48
Daniel               47
                     ..
Noa Greén             1
Lilly Montes          1
thushan madusanka     1
Rp Bautista           1
James Kern            1
Name: userName, Length: 467736, dtype: int64

In [136]:
from sklearn import preprocessing

In [137]:
label_encoder = preprocessing.LabelEncoder()

In [139]:
d7['userName']= label_encoder.fit_transform(d7['userName'])

/Users/udaypratapsingh/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [140]:
d7['userName'].nunique()

467736

In [141]:
d7['userName'].value_counts()

14144     70
74150     57
22234     56
136853    48
88605     47
          ..
444394     1
456684     1
458733     1
452590     1
0          1
Name: userName, Length: 467736, dtype: int64

In [142]:
d7.shape

(660293, 21)

In [143]:
d7.head()

,Unnamed: 0,Activity,Acivity_General,TimeStamp,userName,content,score,thumbsUpCount,Clean Titles,neg,...,pos,compound,nb_chars,nb_words,Samsung Gear Mentioned,Mi Fit Mentioned,FitBit Mentioned,Compound Sentiment,App,dominant_topic
0,0,2 Star Review for Fit Bit,2 Star Review,2021-03-05 00:16:09,183692,"They may have done a critical update, now all ...",2,0,may do critical update get fitbit time need watch,0.223,...,0.000,-0.3182,102,23,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,NaN,Negative,FitBit,10
1,1,5 Star Review for Fit Bit,5 Star Review,2021-03-05 00:12:25,201046,Good app. Very motivating. Great update.,5,0,good app motivate great update,0.000,...,0.828,0.8625,40,6,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,NaN,Positive,FitBit,13
2,2,3 Star Review for Fit Bit,3 Star Review,2021-03-05 00:07:48,437,Always room.for improvement,3,0,always room.for improvement,0.000,...,0.600,0.4588,27,3,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,NaN,Positive,FitBit,4
3,3,1 Star Review for Fit Bit,1 Star Review,2021-03-05 00:04:48,421207,Fix the bug please and don't forget and I can'...,1,3,fix bug please forget can't syce phone fix wor...,0.084,...,0.204,0.4019,201,43,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,NaN,Positive,FitBit,17
4,4,4 Star Review for Fit Bit,4 Star Review,2021-03-05 00:02:57,116184,Works well good inf,4,0,work well good inf,0.000,...,0.714,0.6124,19,4,Samsung Gear Not Mentioned,Mi Fit Not Mentioned,NaN,Positive,FitBit,12


In [148]:
# d7.drop('Unnamed: 0', axis = 1, inplace = True)

In [144]:
d7.to_excel('ABSOLUTE Final KPMG Data with Topics and Encoding - FINAL.xlsx')

In [4]:
d = pd.read_excel('ABSOLUTE Final KPMG Data with Topics and Encoding - FINAL.xlsx', parse_date = True)

In [6]:
d.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)

In [7]:
d.describe(include = 'all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
Activity,660293,18,Reply from Samsung for Samsung,116157,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Acivity_General,660293,8,5 Star Review,212589,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TimeStamp,660293,650243,2017-04-20 07:52:19,5,2012-11-29 17:45:49,2021-03-05 00:16:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
userName,660293,NaN,NaN,NaN,NaT,NaT,232012,134445,0,115917,231798,347340,467735
content,660293,561232,Good,9631,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
score,660293,6,5,212589,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
thumbsUpCount,660293,NaN,NaN,NaN,NaT,NaT,0.878227,10.9506,0,0,0,0,1308
Clean Titles,658229,526231,good,17570,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
neg,660293,NaN,NaN,NaN,NaT,NaT,0.0525181,0.127381,0,0,0,0,1
neu,660293,NaN,NaN,NaN,NaT,NaT,0.419387,0.364487,0,0,0.423,0.72,1
